<a href="https://colab.research.google.com/github/ShaliniR8/google-colab-notebooks/blob/master/RNN_calculator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**IMPORT LIBRARIES**

In [ ]:
import numpy as np
import os
import tensorflow as tf

In [ ]:
dir = ['weights1/', 'model/']
for folder in dir:
  if not os.path.isdir(folder):
    os.mkdir(folder)

In [ ]:
all_chars = '0123456789+'
num_features = len(all_chars)
char_to_index = dict((c,i) for i,c in enumerate(all_chars))
index_to_char = dict((i,c) for i,c in enumerate(all_chars))


**SIMULATE DATA**

In [ ]:
def generate_data():
  first_num = np.random.randint(low = 0, high = 100)
  second_num = np.random.randint(low = 0, high = 100)
  example = str(first_num) + '+' + str(second_num)
  label = str(first_num + second_num)
  return example,label

print(generate_data())

('88+52', '140')


In [ ]:
hidden_units = 128
max_time_steps = 5

model = tf.keras.models.Sequential([
    tf.keras.layers.SimpleRNN(hidden_units, input_shape = (None, num_features)),
    tf.keras.layers.RepeatVector(max_time_steps),
    tf.keras.layers.SimpleRNN(hidden_units, return_sequences=True ),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(num_features, activation='softmax'))
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn (SimpleRNN)       (None, 128)               17920     
_________________________________________________________________
repeat_vector (RepeatVector) (None, 5, 128)            0         
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 5, 128)            32896     
_________________________________________________________________
time_distributed (TimeDistri (None, 5, 11)             1419      
Total params: 52,235
Trainable params: 52,235
Non-trainable params: 0
_________________________________________________________________


**VECTORIZE EXAMPLES**

In [ ]:
def vectorize_example(example, label):
    
    x = np.zeros((max_time_steps, num_features))
    y = np.zeros((max_time_steps, num_features))
    
    diff_x = max_time_steps - len(example)
    diff_y = max_time_steps - len(label)
    #print('diff_x and diff_y:', diff_x, diff_y)
    
    for i, c in enumerate(example):
        x[diff_x+i, char_to_index[c]] = 1
    for i in range(diff_x):
        x[i, char_to_index['0']] = 1
    for i, c in enumerate(label):
        y[diff_y+i, char_to_index[c]] = 1
    for i in range(diff_y):
        y[i, char_to_index['0']] = 1
    
        
    return x, y

e, l = generate_data()
print('Text Example and Label:', e, l)
x, y = vectorize_example(e, l)
print('Vectorized Example and Label Shapes:', x.shape, y.shape)

Text Example and Label: 75+67 142
Vectorized Example and Label Shapes: (5, 11) (5, 11)


**DEVECTORIZE**

In [ ]:
def devectorize_example(example):
    result = [index_to_char[np.argmax(vec)] for vec in example]
    return ''.join(result)

**CREATE DATASET**

In [ ]:
def create_dataset(num_examples=2000):

    x_train = np.zeros((num_examples, max_time_steps, num_features))
    y_train = np.zeros((num_examples, max_time_steps, num_features))

    for i in range(num_examples):
        e, l = generate_data()
        x, y = vectorize_example(e, l)
        x_train[i] = x
        y_train[i] = y
    
    return x_train, y_train

x_train, y_train = create_dataset()
print(x_train.shape, y_train.shape)

(2000, 5, 11) (2000, 5, 11)


In [ ]:
def create_test(num_examples = 500):
  x_test = np.zeros((num_examples, max_time_steps, num_features))
  y_test = np.zeros((num_examples, max_time_steps, num_features))

  for i in range(num_examples):
    e, l = generate_data()
    x, y = vectorize_example(e, l)
    x_test[i] = x
    y_test[i] = y

  return x_test, y_test

x_test, y_test = create_test()
print(x_test.shape, y_test.shape)

(500, 5, 11) (500, 5, 11)


**TRAIN AND SAVE WEIGHTS**

In [ ]:
chk_pt = 'weights1/'

_ = model.fit(
    x_train, y_train,
    validation_data = ( x_test, y_test ),
    epochs = 100,
    batch_size = 64,
    callbacks = [
                 tf.keras.callbacks.ModelCheckpoint(
                     os.path.join( chk_pt, 'epoch_{epoch:02d}_acc_{val_accuracy:.4f}' ),
                     monitor = 'val_accuracy',
                     save_best_only = True,
                     save_weights_only = True
                 )
    ]

)

Epoch 1/100
32/32 [==============================] - 1s 20ms/step - loss: 1.2979 - accuracy: 0.5558 - val_loss: 1.0357 - val_accuracy: 0.6128
Epoch 2/100
32/32 [==============================] - 0s 10ms/step - loss: 1.0123 - accuracy: 0.6168 - val_loss: 0.9939 - val_accuracy: 0.6240
Epoch 3/100
32/32 [==============================] - 0s 10ms/step - loss: 0.9741 - accuracy: 0.6345 - val_loss: 0.9618 - val_accuracy: 0.6488
Epoch 4/100
32/32 [==============================] - 0s 10ms/step - loss: 0.9401 - accuracy: 0.6563 - val_loss: 0.9255 - val_accuracy: 0.6660
Epoch 5/100
32/32 [==============================] - 0s 10ms/step - loss: 0.9056 - accuracy: 0.6726 - val_loss: 0.8938 - val_accuracy: 0.6752
Epoch 6/100
32/32 [==============================] - 0s 10ms/step - loss: 0.8582 - accuracy: 0.6915 - val_loss: 0.8432 - val_accuracy: 0.6872
Epoch 7/100
32/32 [==============================] - 0s 10ms/step - loss: 0.8130 - accuracy: 0.7073 - val_loss: 0.8187 - val_accuracy: 0.7080
Epoch 